# Questions 10-13 of SQL mini-project

Importing relevant packages and creating engine

In [1]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

**Q10:** Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [2]:
pd.read_sql_query('''
    SELECT facility_name, SUM(booking_cost) AS revenue
    FROM (
        SELECT f.name AS facility_name, 
            CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                 ELSE f.membercost * b.slots
            END AS booking_cost
        FROM Bookings as b
        INNER JOIN Facilities as f
            USING (facid)
    ) AS sub
    GROUP BY facility_name
    HAVING revenue < 1000
    ORDER BY revenue DESC;
    ''', engine)

,facility_name,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


**Q11:** Produce a report of members and who recommended them in alphabetic surname,firstname order.

In [3]:
pd.read_sql_query('''
    SELECT m.surname, m.firstname, s.name AS recommended_by,
        m.address, m.zipcode, m.telephone, m.joindate
    FROM Members as m
    LEFT JOIN (
        SELECT memid, surname || ', ' || firstname AS name
        FROM Members
    ) AS s ON m.recommendedby = s.memid;
    ''', engine).head(10)

,surname,firstname,recommended_by,address,zipcode,telephone,joindate
0,GUEST,GUEST,None,GUEST,0,(000) 000-0000,2012-07-01 00:00:00
1,Smith,Darren,None,"8 Bloomsbury Close, Boston",4321,555-555-5555,2012-07-02 12:02:05
2,Smith,Tracy,None,"8 Bloomsbury Close, New York",4321,555-555-5555,2012-07-02 12:08:23
3,Rownam,Tim,None,"23 Highway Way, Boston",23423,(844) 693-0723,2012-07-03 09:32:15
4,Joplette,Janice,"Smith, Darren","20 Crossing Road, New York",234,(833) 942-4710,2012-07-03 10:25:05
5,Butters,Gerald,"Smith, Darren","1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,2012-07-09 10:44:09
6,Tracy,Burton,None,"3 Tunisia Drive, Boston",45678,(822) 354-9973,2012-07-15 08:52:55
7,Dare,Nancy,"Joplette, Janice","6 Hunting Lodge Way, Boston",10383,(833) 776-4001,2012-07-25 08:59:12
8,Boothe,Tim,"Rownam, Tim","3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,2012-07-25 16:02:35
9,Stibbons,Ponder,"Tracy, Burton","5 Dragons Way, Winchester",87630,(833) 160-3900,2012-07-25 17:09:05


**Q12:** Find the facilities with their usage by member, but not guests.

In [4]:
pd.read_sql_query('''
    SELECT name AS facility_name, COUNT(*) as total_mem_bookings
    FROM (
        SELECT b.memid, f.name
        FROM Bookings AS b
        INNER JOIN Facilities as f USING (facid)
        WHERE b.memid != 0
    ) as sub
    GROUP BY name;
    ''', engine).head(20)

,facility_name,total_mem_bookings
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


**Q13:** Find the facilities usage by month, but not guests.

In [5]:
pd.read_sql_query('''
    SELECT
        CASE 
            WHEN month = '07' THEN 'July'
            WHEN month = '08' THEN 'August'
            WHEN month = '09' THEN 'September'
        END AS month,
        name AS facility_name, COUNT(*) AS mem_bookings
    FROM (
        SELECT strftime('%m', b.starttime) AS month, f.name
        FROM Bookings as b
        INNER JOIN Facilities AS f USING (facid)
        WHERE b.memid !=0) AS sub
    GROUP BY month, name;
    ''', engine)

,month,facility_name,mem_bookings
0,July,Badminton Court,51
1,July,Massage Room 1,77
2,July,Massage Room 2,4
3,July,Pool Table,103
4,July,Snooker Table,68
5,July,Squash Court,23
6,July,Table Tennis,48
7,July,Tennis Court 1,65
8,July,Tennis Court 2,41
9,August,Badminton Court,132
